## 빠른 시작
이번 장에서는 머신 러닝의 일반적인 작업들을 위한 API를 통해 실행됩니다. 더 자세히 알아보려면 각 장의 링크를 참고하세요

### 데이터 작업하기
파이토치에서는 데이터 작업을 위한 기본 요소 두가지인
`torch.utils.data.DataLoader`와 `torch.utils.data.Dataset`가 있습니다. `Dataset`은 *샘플과 정답(Label)* 을 저장하고, `DataLoader`는 `Dataset`을 순회 가능한 객체(Iterable)로 감쌉니다.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

C:\Users\LUDOBICO\AppData\Roaming\Python\Python37\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pytorch는 `TorchText, TorchVision` 및 `TorchAudio`와 같이 도메인 특화 라이브러리를 데이터셋과 함께 제공하고 있습니다. 이 튜토리얼에서는 TorchVision 데이터셋을 사용하도록 하겠습니다.

`torchvision.datasets` 모듈은 CIFAR, COCO 등과 같은 다양한 실제 비전(vision) 데이터에 대한 Dataset을 포함하고 있습니다. 이 튜토리얼에서는 FasionMNIST 데이터셋을 사용합니다. 모든 TorchVision dataset은 샘플과 정답을 각각 변경하기 위한 `transform`과 `target_transform` 의 두 인자를 포함합니다.

In [2]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.

training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:24<00:00, 1061346.57it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 107674.78it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:03<00:00, 1343708.62it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



`Dataset`을 `DataLoader` 의 인자로 전달합니다. 이는 데이터셋을 순화 가능한 객체로 감싸고, 자동화된 배치(batch), 샘플링(sampling), 섞기(shuffle) 및 다중 프로세스로 데이터 불러오기 (multiprocess data laoding)를 지원합니다. 여기서는 배치 크기(batch size)를 64로 정의합니다. 즉, 데이터로더(dataloader) 객체의 각 요소는 64개의 특징과 정답을 묶음(batch)로 반환합니다.

In [4]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for x,y in test_dataloader:
    print('shape of x [N,C,H,W]: {0}'.format(x.shape))
    print('shape of y: {0}'.format(y.dtype))
    break


shape of x [N,C,H,W]: torch.Size([64, 1, 28, 28])
shape of y: torch.int64


## 모델 만들기
Pytorch에서 신경망 모델은 `nn.Module`을 상속받는 클래스(class)를 생성하여 정의합니다. `__init__` 함수에서 신견망의 계층(layer)들을 정의하고, `forward` 함수에서 신경망에 데이터를 어떻게 전달할지 지정합니다. 가능한 경우 GPU로 신경망을 이동시켜 연산을 가속(acclerate)합니다.

In [18]:
# 학습에 사용할 cpu나 gpu 장치를 얻습니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using {0}'.format(device))

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

using cuda


In [11]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=18, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부 백그라운드 연산들과 함께 모델의 `forward` 를 실행합니다.
`model.forward()`를 직접 호출하지 마세요

모델에 입력을 호출하면 각 분류(class)에 대한 원시(raw) 예측값이 있는 10차원 텐서가 반환됩니다. 원시 예측값을 `nn.softmax` 모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.

In [21]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)

pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print('Predicted class: {0}'.format(y_pred))

NotImplementedError: Module [NeuralNetwork] is missing the required "forward" function